# Q1. Running ElasticSearch

### What's the version.build_hash value?

**Answer**

In [2]:
!curl localhost:9200

{
  "name" : "6e4c5090ddd2",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "o_4rzr-sRES8MgBv7E8v5Q",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


`build_hash` value is:
```
42f05b9372a9a4a470db3b52817899b99a76ee73
```

# Q2. Indexing the data

In [3]:
import requests 
import json

documents = []
docs_url = 'documents.json'

with open(docs_url, 'rt') as f_in:
    documents_raw = json.load(f_in)

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

print(documents[0])

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-zoomcamp'}


### Index the data in the same way as was shown in the course videos. Make the course field a keyword and the rest should be text.

In [ ]:
from elasticsearch import Elasticsearch

# create Elasticsearch client
es = Elasticsearch("http://localhost:9200")

# create mappings with expected types
mappings = {
    "properties": {
        "text": {"type": "text"},
        "section": {"type": "text"},
        "question": {"type": "text"},
        "course": {"type": "keyword"} # make coursefield keyword
    }
}

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": mappings
}

In [6]:
index_name = "course-questions"

# create an index with mappings
es.indices.create(index=index_name, body=index_settings)

# index over each document
for doc in documents:
    es.index(index=index_name, document=doc)

### Which function do you use for adding your data to elastic?

**Answer**

```
index()
```

# Q3. Searching

In [7]:
query = "How do I execute a command in a running docker container?"

In [9]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"], # Use only question and text fields, with a boost of 4 to question
                    "type": "best_fields"
                }
            }
        }
    }
}

In [10]:
results = es.search(
    index=index_name,
    body=search_query    
)

### What's the score for the top ranking result?

**Answer**

In [16]:
top_result = results['hits']['hits'][0]
top_result['_score']

84.050095

# Q4. Filtering

Now let's only limit the questions to `machine-learning-zoomcamp`.

### Return 3 results. What's the 3rd question returned by the search engine?

In [22]:
search_query = {
        "size": 3, # return 3 results
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp" # filter on ml-zoomcamp
                    }
                }
            }
        }
    }

**Answer**

In [25]:
results = es.search(
    index=index_name,
    body=search_query    
)
third_result = results['hits']['hits'][2]
third_result['_source']['question']

'How do I copy files from a different folder into docker container’s working directory?'

# Q5. Building a prompt

Take the records returned from Elasticsearch in Q4 and use this template to build the context.

In [37]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [40]:
context = ""

for result in results['hits']['hits']:
    context += context_template.format(
        question=result['_source']["question"], 
        text=result['_source']["text"]
    ).strip() + "\n\n"

In [41]:
context

'Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani\n\nQ: How do I copy files from a different folder into docker container’s working directory?\nA: You can copy files from your local machine into a Docker container using the docker cp comman

In [42]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt = prompt_template.format(question=query, context=context).strip()

In [43]:
prompt

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do I execute a command in a running docker container?\n\nCONTEXT:\nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_dir

### What's the length of the resulting prompt? (use the len function)

**Answer**

In [44]:
len(prompt)

1462

# Q6. Tokens

In [45]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 37.0 MB/s eta 0:00:00


In [47]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

### Use the encode function. How many tokens does our prompt have?

**Answer**

In [49]:
len(encoding.encode(prompt))

322

# Q7. Using OpenAI

In [53]:
import os 
os.environ["OPENAI_API_KEY"] = "<my-super-secret-key>"

In [54]:
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

In [55]:
response

ChatCompletion(id='chatcmpl-BGaK5k1cFJFtiiquGbE1qN0oKYJLH', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='To execute a command in a running Docker container, use the `docker exec` command. First, find the container ID by using `docker ps`. Then, execute the command in the specific container with the following syntax:\n\n```\ndocker exec -it <container-id> command\n```\n\nFor example, to start a bash session inside the container, use:\n\n```\ndocker exec -it <container-id> bash\n```', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1743292573, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_6dd05565ef', usage=CompletionUsage(completion_tokens=86, prompt_tokens=329, total_tokens=415, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens